## Multi-class classification

> Indented block



Iris dataset https://en.wikipedia.org/wiki/Iris_flower_data_set

In [ ]:
import numpy as np
from tensorflow import summary
import datetime, os
import math
import random
import torch
import matplotlib.pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix

## For this experiment we will use PyTorch framework

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

#### Experiments logging via tensorboard

In [ ]:
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
train_summary_writer = summary.create_file_writer(logdir)
validation_summary_writer = summary.create_file_writer(logdir)

In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=16530d7969188ebbc2aff0ea7e56eeda3b740cfecb246f3b71e9dfabc3728363
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


#### Check if GPU is available

In [ ]:
!nvidia-smi

Thu Nov 24 17:13:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |    312MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# nuskaitome duomenis
data = np.genfromtxt("./2_iris.csv", delimiter='\t') 

# paskirstome apmokymas, validavimas, testavimas
N_samples = data.shape[0]
N_train = int(N_samples * 0.7) # 70% apmokymas
N_val = int(N_samples * 0.8) # 10% validavimas
N_test = N_samples - N_train - N_val

In [ ]:
N_samples

150

In [ ]:
data

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

In [ ]:

# randomizuojame stebėjimus
shuffle_data = [x for x in np.random.permutation(N_samples)]
data = data[shuffle_data,:]


In [ ]:
data

array([[5.2, 3.4, 1.4, 0.2, 0. ],
       [4.6, 3.2, 1.4, 0.2, 0. ],
       [6.8, 2.8, 4.8, 1.4, 1. ],
       [4.8, 3.1, 1.6, 0.2, 0. ],
       [4.4, 3.2, 1.3, 0.2, 0. ],
       [5.9, 3.2, 4.8, 1.8, 1. ],
       [5.1, 3.5, 1.4, 0.2, 0. ],
       [6.7, 3.1, 4.4, 1.4, 1. ],
       [5.6, 2.5, 3.9, 1.1, 1. ],
       [5.1, 3.8, 1.6, 0.2, 0. ],
       [6.7, 3. , 5. , 1.7, 1. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [5.5, 2.6, 4.4, 1.2, 1. ],
       [6.6, 3. , 4.4, 1.4, 1. ],
       [6.3, 3.3, 4.7, 1.6, 1. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [5.5, 3.5, 1.3, 0.2, 0. ],
       [6.1, 2.9, 4.7, 1.4, 1. ],
       [6.2, 2.9, 4.3, 1.3, 1. ],
       [5.7, 2.8, 4.1, 1.3, 1. ],
       [6. , 2.7, 5.1, 1.6, 1. ],
       [6. , 3.4, 4.5, 1.6, 1. ],
       [5.5, 4.2, 1.4, 0.2, 0. ],
       [5. , 3.3, 1.4, 0.2, 0. ],
       [5.6, 2.7, 4.2, 1.3, 1. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [6.7, 3.1, 4.7, 1.5, 1. ],
       [5.1, 3.4, 1.5, 0.2, 0. ],
       [6.7, 3

In [ ]:
## fiktyvių kintamųjų sukūrimas
def batch_to_one_hot(id, batch_size, nb_classes = 10):
    y_onehot = torch.FloatTensor(batch_size, nb_classes)
    y_onehot.zero_()
    y_onehot.scatter_(1, id.view(-1, 1), 1)
    return y_onehot.long()

In [ ]:

# Duomenų skaidymas -> test, validation, test
data_train = torch.tensor(data[:N_train,:4]).float()
y_train = torch.tensor(data[:N_train,4], dtype= torch.int64)
y_train = batch_to_one_hot(y_train, y_train.shape[0], 3)
data_train.shape

data_val = torch.tensor(data[N_train:N_val,:4]).float()
y_val = torch.tensor(data[N_train:N_val,4], dtype= torch.int64)
y_val = batch_to_one_hot(y_val, y_val.shape[0], 3)
data_val.shape

data_test = torch.tensor(data[N_val:,:4]).float()
y_test = torch.tensor(data[N_val:,4], dtype= torch.int64)
y_test = batch_to_one_hot(y_test, y_test.shape[0], 3)
data_test.shape

torch.Size([30, 4])

In [ ]:
y_train

tensor([[1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.bn1 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 15)
        self.bn2 = nn.BatchNorm1d(15)
        self.fc3 = nn.Linear(15, 3)
        self.softmax = nn.Softmax()
        self.drop = nn.Dropout(p=0.4)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.drop(X)
        X = self.bn1(X)
        X = self.fc2(X)
        X = self.drop(X)
        X = self.bn2(X)
        X = self.fc3(X)
        X = F.softmax(X)

        return X

model = NeuralNet()

In [ ]:
# apibrėžiame nuostolių f-ją
a1 = 1
a2 = 1
loss_func = a1 * nn.CrossEntropyLoss() #+ a2 * nn.customLoss()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss_func(input, target)
input, target

(tensor([[ 1.0158,  1.4704, -0.5244,  0.5490,  0.1420],
         [-0.1848, -1.2149, -0.9978,  0.3065,  1.4839],
         [-0.3748,  0.0056,  0.6661,  0.1335, -1.1682]], requires_grad=True),
 tensor([0, 1, 3]))

In [ ]:

# pasirenkame optimizatorių
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# iteracijų (epochų skaičius) kiek ilgai iteraciškai vertinsime parametrus
n_epochs = 10

for epoch in range(0, n_epochs+1):

    # pirmą epochą netreniruosim, kad tensorboard turėtų pradinį tašką
    if epoch == 0:
      torch.set_grad_enabled(False)

    # stebėjime tikslo f-jos pokytį
    train_loss = 0.0
    validation_loss = 0.0
    train_acc = 0.0
    validation_acc = 0.0
    
    #########################################################
    # Modelio parametrų vertinimas (angl. training)          #
    #########################################################

    # nunuliname preitos operacijos gradientą
    optimizer.zero_grad()
    # Praleidžiame duomenis X pro modelį, gauname prognozę yhat
    outputs = model(data_train)
    
    # apskaičiuojam tikslumą
    with torch.no_grad():
      outputLabels = torch.argmax(outputs, 1)
      yLabels = torch.argmax(y_train, 1)
      accuracy = yLabels == outputLabels
      train_acc += torch.sum(accuracy.float())

    # skaičiuojame, nuostolius, kiek prognozė yhat, skiriasi nuo tiktų reikšmių
    loss = loss_func(outputs, yLabels)

    # apskaičiuojame gradientų reikšmes, visiems nežinomiems parametrams
    if epoch != 0:
      loss.backward()
    # atnaujiname, visus nežinomus, modelio parametrus
    optimizer.step()
    # skaičiuojame bendrą nuostolį
    train_loss += loss.item()*data_train.size(0)
        
        
    
    ##############################################################
    # Modelio kokybės vertinimas, mokymo metu (angl. validation) #
    ##############################################################
    # pasiimame duomenų rinkinio poaibį (X, y)
    # Praleidžiame duomenis X pro modelį, gauname prognozę yhat
    outputs = model(data_val)
    # skaičiuojame, nuostolius, kiek prognozė yhat, skiriasi nuo tiktų reikšmių
    yLabels = torch.argmax(y_val, 1)
    loss = loss_func(outputs, yLabels)
    # skaičiuojame bendrą nuostolį
    validation_loss += loss.item()*data_val.size(0)

    # skaičiuojame bendrą tikslumą
    with torch.no_grad():
      outputLabels = torch.argmax(outputs, 1)
      accuracy = yLabels == outputLabels
      validation_acc += torch.sum(accuracy.float())

    # validacijos metu, nes nekeičiame parametrų, tik pasižiūrime, kaip modelis 
    # mokinimo metu veikia ant nematytų duomenų
        
    with train_summary_writer.as_default():
        summary.scalar('ConvLoss/train', train_loss / N_train, step=epoch)
        summary.scalar('ConvAcc/train', train_acc / N_train, step=epoch)
        
    with validation_summary_writer.as_default():
        summary.scalar('ConvLoss/validation', validation_loss / N_val, step=epoch)
        summary.scalar('ConvAcc/validation', validation_acc / N_val, step=epoch)
            
    # spausdiname tarpines statistikas
    print('Epocha: {} \t Nuostoliai: {:.10f} \t Tikslumas: {:.3f} \t Validacijos nuostoliai: {:.10f} \t Validacijos tikslumas: {:.3f}'.format(
        epoch, 
        train_loss / N_train,
        train_acc / N_train,
        validation_loss / N_val,
        validation_acc / N_val,
        ))
    
    if epoch == 0:
      torch.set_grad_enabled(True)

test_loss = 0.0
################################################################
# Nepriklausomas modelio testavimas, po mokymo (angl. testing) #
################################################################
# pasiimame duomenų rinkinio poaibį (X, y)

# Praleidžiame duomenis X pro modelį, gauname prognozę yhat
outputs = model(data_test)
# skaičiuojame bendrą nuostolį
yLabels = torch.argmax(y_test, 1)
loss = loss_func(outputs, yLabels)
# skaičiuojame bendrą nuostolį
test_loss += loss.item()*data_test.size(0)

print('Epocha: {} \t Testavimo nuostoliai (Nematyti duomenys): {:.10f}'.format(
    epoch, 
    test_loss / N_test
    ))

Epocha: 0 	 Nuostoliai: 1.0701084137 	 Tikslumas: 0.438 	 Validacijos nuostoliai: 0.1346009374 	 Validacijos tikslumas: 0.042
Epocha: 1 	 Nuostoliai: 1.0532292128 	 Tikslumas: 0.486 	 Validacijos nuostoliai: 0.1331671625 	 Validacijos tikslumas: 0.067
Epocha: 2 	 Nuostoliai: 1.0883837938 	 Tikslumas: 0.390 	 Validacijos nuostoliai: 0.1293273866 	 Validacijos tikslumas: 0.075
Epocha: 3 	 Nuostoliai: 1.0344611406 	 Tikslumas: 0.505 	 Validacijos nuostoliai: 0.1305756420 	 Validacijos tikslumas: 0.050
Epocha: 4 	 Nuostoliai: 1.0369391441 	 Tikslumas: 0.524 	 Validacijos nuostoliai: 0.1289289743 	 Validacijos tikslumas: 0.058
Epocha: 5 	 Nuostoliai: 1.0302056074 	 Tikslumas: 0.562 	 Validacijos nuostoliai: 0.1316546947 	 Validacijos tikslumas: 0.058
Epocha: 6 	 Nuostoliai: 1.0296562910 	 Tikslumas: 0.533 	 Validacijos nuostoliai: 0.1309192330 	 Validacijos tikslumas: 0.058
Epocha: 7 	 Nuostoliai: 1.0225884914 	 Tikslumas: 0.562 	 Validacijos nuostoliai: 0.1319069117 	 Validacijos tikslumas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
outputs

tensor([[0.2158, 0.3036, 0.4806],
        [0.3501, 0.3570, 0.2930],
        [0.3157, 0.2041, 0.4802],
        [0.5237, 0.3137, 0.1627],
        [0.1505, 0.4419, 0.4076],
        [0.6920, 0.0906, 0.2174],
        [0.0409, 0.5773, 0.3818],
        [0.1947, 0.2115, 0.5938],
        [0.7371, 0.0094, 0.2535],
        [0.3208, 0.3627, 0.3165],
        [0.4012, 0.2447, 0.3541],
        [0.2756, 0.4452, 0.2792],
        [0.2189, 0.6526, 0.1285],
        [0.3754, 0.2448, 0.3798],
        [0.4497, 0.2686, 0.2818],
        [0.8802, 0.0722, 0.0475],
        [0.1130, 0.3178, 0.5693],
        [0.9224, 0.0048, 0.0727],
        [0.7800, 0.0933, 0.1267],
        [0.3455, 0.2948, 0.3597],
        [0.1759, 0.4884, 0.3357],
        [0.2158, 0.3586, 0.4256],
        [0.4743, 0.2292, 0.2965],
        [0.1327, 0.4921, 0.3751],
        [0.3249, 0.4549, 0.2203],
        [0.5366, 0.1930, 0.2703],
        [0.1341, 0.5541, 0.3118],
        [0.1389, 0.5454, 0.3157],
        [0.1561, 0.2033, 0.6406],
        [0.153

In [ ]:

# test model on unseen (test) data
predictions = model(data_test)
predictions
predicted_classes = torch.argmax(predictions, axis = 1)
predicted_classes
predicted = batch_to_one_hot(predicted_classes, predictions.shape[0], 3)
predicted
multilabel_confusion_matrix(y_test, predicted)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


array([[[17,  8],
        [ 0,  5]],

       [[ 9,  7],
        [10,  4]],

       [[16,  3],
        [ 8,  3]]])

In [ ]:
predicted

tensor([[0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0]])

In [ ]:
predictions # this is - predicted values y hat 

tensor([[0.2005, 0.2416, 0.5579],
        [0.2316, 0.3069, 0.4616],
        [0.4738, 0.2545, 0.2717],
        [0.2056, 0.4686, 0.3257],
        [0.4092, 0.2090, 0.3818],
        [0.7998, 0.1346, 0.0655],
        [0.4274, 0.1051, 0.4675],
        [0.1069, 0.5359, 0.3572],
        [0.8757, 0.0519, 0.0724],
        [0.2199, 0.3699, 0.4102],
        [0.1964, 0.5866, 0.2170],
        [0.4745, 0.0914, 0.4340],
        [0.3267, 0.5206, 0.1527],
        [0.3675, 0.2829, 0.3496],
        [0.3466, 0.3241, 0.3293],
        [0.6172, 0.1741, 0.2087],
        [0.0719, 0.5400, 0.3881],
        [0.9335, 0.0033, 0.0632],
        [0.4074, 0.2474, 0.3452],
        [0.3019, 0.2122, 0.4860],
        [0.5542, 0.1139, 0.3319],
        [0.0608, 0.8351, 0.1041],
        [0.2658, 0.3828, 0.3514],
        [0.1189, 0.6326, 0.2485],
        [0.5179, 0.1876, 0.2945],
        [0.3360, 0.3772, 0.2868],
        [0.0993, 0.4950, 0.4057],
        [0.1326, 0.1160, 0.7514],
        [0.5085, 0.1715, 0.3200],
        [0.138

## Error analysis

In [ ]:
predictions

tensor([[0.2005, 0.2416, 0.5579],
        [0.2316, 0.3069, 0.4616],
        [0.4738, 0.2545, 0.2717],
        [0.2056, 0.4686, 0.3257],
        [0.4092, 0.2090, 0.3818],
        [0.7998, 0.1346, 0.0655],
        [0.4274, 0.1051, 0.4675],
        [0.1069, 0.5359, 0.3572],
        [0.8757, 0.0519, 0.0724],
        [0.2199, 0.3699, 0.4102],
        [0.1964, 0.5866, 0.2170],
        [0.4745, 0.0914, 0.4340],
        [0.3267, 0.5206, 0.1527],
        [0.3675, 0.2829, 0.3496],
        [0.3466, 0.3241, 0.3293],
        [0.6172, 0.1741, 0.2087],
        [0.0719, 0.5400, 0.3881],
        [0.9335, 0.0033, 0.0632],
        [0.4074, 0.2474, 0.3452],
        [0.3019, 0.2122, 0.4860],
        [0.5542, 0.1139, 0.3319],
        [0.0608, 0.8351, 0.1041],
        [0.2658, 0.3828, 0.3514],
        [0.1189, 0.6326, 0.2485],
        [0.5179, 0.1876, 0.2945],
        [0.3360, 0.3772, 0.2868],
        [0.0993, 0.4950, 0.4057],
        [0.1326, 0.1160, 0.7514],
        [0.5085, 0.1715, 0.3200],
        [0.138

In [ ]:
np.log(predictions.detach().numpy()).shape

(30, 3)

In [ ]:
y_test.detach().numpy().shape

(30, 3)

In [ ]:
error = -np.sum(y_test.detach().numpy()*np.log(predictions.detach().numpy()),axis=1)
error

array([0.58359981, 1.18138182, 1.36831295, 0.75794369, 1.56521952,
       0.22333735, 0.7602666 , 1.02948475, 0.13269223, 0.99448502,
       0.53338772, 2.39200044, 0.65268201, 1.26252484, 1.12657714,
       0.48257583, 0.9465549 , 0.06881189, 0.89789498, 1.55035508,
       2.17227077, 2.26233268, 1.04583776, 1.39213955, 1.67364085,
       0.9749949 , 0.90205491, 0.28586519, 1.13933468, 1.35487437])

In [ ]:
err_idx = np.argsort(error)
err_idx

array([17,  8,  5, 27, 15, 10,  0, 12,  3,  6, 18, 26, 16, 25,  9,  7, 22,
       14, 28,  1, 13, 29,  2, 23, 19,  4, 24, 20, 21, 11])

In [ ]:
y_test[err_idx], error[err_idx]

(tensor([[1, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [0, 0, 1],
         [1, 0, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 0, 1],
         [1, 0, 0],
         [0, 0, 1],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 0, 1],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 1, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0]]),
 array([0.06881189, 0.13269223, 0.22333735, 0.28586519, 0.48257583,
        0.53338772, 0.58359981, 0.65268201, 0.75794369, 0.7602666 ,
        0.89789498, 0.90205491, 0.9465549 , 0.9749949 , 0.99448502,
        1.02948475, 1.04583776, 1.12657714, 1.13933468, 1.18138182,
        1.26252484, 1.35487437, 1.36831295, 1.39213955, 1.55035508,
        1.56521952, 1.67364085, 2.17227077, 2.26233268, 2.